In [ ]:
# %%capture --no-stderr
# !pip install git+https://github.com/openai/CLIP timm

In [1]:
import torch
import numpy as np
from PIL import Image
import cv2 as cv
import matplotlib.pyplot as plt
from tqdm import tqdm

from omegaconf import OmegaConf
from igniter.builder import build_engine
from fsl.models.meta_arch import build_fsod

plt.rcParams['figure.figsize'] = [10, 8]

In [2]:
config_file = '../configs/devit/resnet_trainval_30shot.yaml'
# config_file = '../configs/devit/sam_vitb_trainval_30shot.yaml'

cfg = OmegaConf.load(config_file)
# print(OmegaConf.to_yaml(cfg))

cfg.datasets.dataloader.batch_size = 1
cfg.datasets.dataloader.num_workers = 0
cfg.datasets.dataloader.shuffle=False
cfg.options.train = False
cfg.options.eval = True
cfg.build.resnet_fsod.weights="s3://sr-shokunin/perception/models/fsl/resnet_vitb_coco17_30shot/model_0000019.pt"
# cfg.build.sam_fsod.weights = "s3://sr-shokunin/perception/models/fsl/sam_vitb_coco17_30shot/model_0000001.pt"

engine = build_engine(cfg)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-09 18:02:01,135 [builder.py:249] WARNING: # TODO: Remove hardcoded name and replace with registry based
/root/krishneel/Documents/github/igniter/igniter/datasets/s3_dataset.py:55: UserWarning: Target transforms is not yet implemented
  warnings.warn('Target transforms is not yet implemented')


loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [4]:
size = max([len(iid) for iid in engine._model.classifier._all_cids]) + 1
gt_labels = lambda indices: [engine._model.classifier._all_cids[i] for i in indices]

def accuracy(scores, gt_names, topk = 1):
    _, indices = torch.topk(scores, topk, dim=1)
    pred_labels = [gt_labels(index) for index in indices]

    count = 0.0
    for gt_name, labels in zip(gt_names, pred_labels):
        count += 1 if gt_name in labels else 0
    return count / len(gt_names)

def print_labels(scores, targets, k = 2):
    gt_names = targets[0]['gt_proposal'].labels
    _, indexes = torch.topk(scores, k, dim=1)
    
    for name, i in zip(gt_names, indexes.cpu().numpy()):
        string = name.ljust(size) + " | " + "".join([n.ljust(size) for n in gt_labels(i)])
        print(string)

In [ ]:
acc_top1, acc_top5 = 0.0, 0.0
progress_bar = tqdm(total=len(engine._dataloader), desc="Processing", position=0, leave=True)

for k, (images, targets) in enumerate(engine._dataloader, 1):
    with torch.no_grad():
        output, _ = engine._model(images, targets)
    scores = output['scores']

    gt_names = targets[0]['gt_proposal'].labels
    t1 = accuracy(scores, gt_names, topk=1)
    t5 = accuracy(scores, gt_names, topk=5)

    acc_top1 += (t1 - acc_top1) / k
    acc_top5 += (t5 - acc_top5) / k

    # print("\nImage ID: ", targets[0]['gt_proposal'].image_id)
    # print_labels(scores, targets, k=5)
    progress_bar.set_description(f"Top 1: {acc_top1:.3f} | Top 5 {acc_top5:.3f}")
    progress_bar.update(1)

progress_bar.close()

print("\nTop 1: ", acc_top1, "Top 5: ", acc_top5)

Top 1: 0.448 | Top 5 0.673:  60%|██████████████████████████████████▊                       | 2505/4167 [18:05<10:02,  2.76it/s]